In [1]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch

# Set up S3 bucket and paths to input data and model artifacts
bucket_name = "store-objects-detection"
prefix = "sagemaker/SKU110K"

training_data_path = f's3://{bucket_name}/{prefix}'
model_artifacts_path = f's3://{bucket_name}/model-artifacts/'

# Set up SageMaker session, role, and instance types
sagemaker_session = sagemaker.Session()
train_instance_type = 'ml.g4dn.xlarge'

role = "sagemaker-iam-role"

In [2]:
from sagemaker.image_uris import retrieve 
training_image = retrieve("pytorch", 
                            region="eu-central-1", 
                            version="1.13.1", 
                            py_version="py39", 
                            instance_type=train_instance_type, 
                            image_scope="training")
print(training_image)

763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39


In [3]:
# Define the training job
estimator = PyTorch(
    image_uri=training_image,
    source_dir="code",
    entry_point="yolo.py",
    role=role,
    py_version="py39",
    framework_version="1.13.1",
    instance_count=1,
    instance_type=train_instance_type,
    output_path=model_artifacts_path,
    sagemaker_session=sagemaker_session,
    hyperparameters = {'epochs': 30, 'batch-size': 4, 'model': 'Faster_RCNN', 'sagemaker': True}
)
estimator.fit({'train': f's3://{bucket_name}/{prefix}',
                'test': f's3://{bucket_name}/{prefix}'}, logs="All")

S3UploadFailedError: Failed to upload /var/folders/qq/63cdqnts00j2c7vpvwcqxnwm0000gn/T/tmp71k_wz40/source.tar.gz to store-objects-detection/pytorch-training-2023-06-06-23-02-18-762/source/sourcedir.tar.gz: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied